In [29]:
import numpy as np
import deepSI
from deepSI import System_data
from deepSI import load_system_data
from deepSI import System_data_list
out= np.load('training-data.npz')
th_train = out['th']
u_train = out['u']

data = np.load('test-prediction-submission-file1.npz')
u_test = data['upast']
th_test = data['thpast']

system_data = System_data(u=u_train,y=th_train) 


if False: #train model
    sys = deepSI.fit_systems.SS_encoder(nx=13,na=2,nb=3) #load the encoder system
    sys.n_hidden_layers = 1
    sys.n_nodes_per_layer = 20
    sys.fit(system_data, val_sys_data=system_data[-12_000:], epochs=500, batch_size=1024, loss_kwargs=dict(nf=40)) 
    sys.save_system('Model_encoder\\encoder')

    
    sim_val = system_data[-25_000:] #larger validation set
    sys.bestfit = sys.apply_experiment(sim_val).NRMS(sim_val) #reset checkpoint
    #use all the data for final local minima search
    sys.fit(system_data,sim_val, epochs=200 ,batch_size=1024, loss_kwargs=dict(nf=40)) 
    sys.save_system('Model_encoder\\encoder_final')
    
else: #load model from file
    sys = deepSI.load_system('Model_encoder\encoder_final')

prediction = sys.multi_step_ahead(system_data, nf=1)   #One step ahead prediction with nf=1, for some reason the one step ahead function in deepSI would not work


print('train prediction errors:')
print('RMS:', prediction.RMS(system_data),'radians')
print('RMS:', prediction.RMS(system_data)/(2*np.pi)*360,'degrees')
print('NRMS:', prediction.NRMS(system_data),'%')

result=[]
for i in range(1366):
    u_test1 = u_test[i,:]
    th_test1 = th_test[i,:]
    system_data1 = System_data(u=u_test1,y=th_test1)
    sys = deepSI.load_system('Model_encoder\encoder_final')
    test_sim = sys.multi_step_ahead(system_data1,nf=1)
    if result==None:
        result = test_sim.y[14]
    else:
        result.append(test_sim.y[14])

np.savez('test_prediction-submission-file.npz', upast=u_test, thpast=th_test, thnow=result)


train prediction errors:
RMS: 0.004673394585040686 radians
RMS: 0.267765785722124 degrees
NRMS: 0.009528418040858527 %
